# Importando las Librerias en principio mas necesarias para este primer paso de generar un Tablon.

Este Tablon estará compuesto por diferentes Dataframes, que deberemos "merger" en uno solo.

In [238]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
%matplotlib inline

# Nos hemos descargado todos los archivos csv disponibles en opendata.euskadi.eus de Indices de Sostenibilidad 

Por ejemplo aqui tendriamos Datos de accesibilidad (cuan accesibles son los municipios vascos)

In [239]:
accesibilidad = '/home/vant/Escritorio/TFM/accesibilidad.csv'

## Leyendo el archivo y comprobando el numero de lineas de este (una prueba con lineas de comando para no perder la constumbre de uso). No son archivos muy pesados como veremos

In [240]:
%time

!cat {accesibilidad} |wc -l

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs
284


## Viendo y comprobando el archivo con la linea de comandos. Warning! tiene lineas de texto con descripcion
Las columnas son temporales y van desde el año 2005 al 2009
Vamos a tener que seleccionar las lineas validas para montar el Dataframe, que va a estar compuesto por municipios.
Hay que contar las lineas que son solo municipios para poder componer el Dataframe correctamente: -251

In [241]:
!cat {accesibilidad} | tail -1

Creando el primer dataframe con pandas (hay un problema de unicode decode error utf-8, lo cual salvamos utilizando un encoding='latin-1')

In [242]:
pd.options.display.max_columns=None

df_accesibilidad = pd.read_csv(accesibilidad, sep=';', encoding='latin-1', skiprows= 33)

## El primer dataframe ya da un problema: no hay informacion de Vitoria-gasteiz, cuando mergamos en inner, no saldra

In [243]:
#Codigo de Vitoria = 1059
df_no_vitoria = df_accesibilidad[df_accesibilidad['Código municipio'] == 1059] 
df_no_vitoria

,Código municipio,Municipio,2009,2008,2007,2006,2005


## Vemos como los datos difieren entre Dataframes, no solo a veces va a faltar algun municipio sino que los años con informacion no son siempre los mismos 


In [244]:
accidentes_trafico = '/home/vant/Escritorio/TFM/accidentetrafic.csv'
df_accident_trafico = pd.read_csv(accidentes_trafico, sep=';', encoding='latin-1', skiprows= 33)
df_accident_trafico.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"14,08","10,90","11,56","10,03","7,49","10,24","10,89","7,66","8,65","8,71","9,62","10,21","13,71","13,28","15,26"
1,20001,Abaltzisketa,"3,07","6,12","6,12","21,24","3,03","6,12","3,10","15,43","0,00","9,46","6,43","3,33","3,45","3,55","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"11,51","9,27","10,83","9,15","12,65","10,07","12,05","11,42","9,60","13,19","13,63","13,95","16,32","17,22","21,18"


## Como se ve en el siguiente Dataframe tambien hay NANs, habra que rellenar la informacion más adelante

In [245]:
calidad_agua_consumo = '/home/vant/Escritorio/TFM/aguayaire/Calificacion sanitaria del agua de consumo.csv'
df_calidad_agua_consumo = pd.read_csv(calidad_agua_consumo, sep=';', encoding='latin-1', skiprows= 4)
df_calidad_agua_consumo[220:223]

,Código municipio,Municipio,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
220,48074,Urduña/Orduña,NaN,NaN,NaN,NaN,NaN,"2,00","2,00","3,00","0,00","3,00","3,00","0,00","0,00","0,00"
221,1054,Urkabustaiz,"3,00","3,00","3,00","2,00","3,00","3,00","3,00","1,00","1,00","2,00","2,00","1,00","3,00","3,00"
222,20072,Urnieta,"3,00","3,00","2,00","2,00","3,00","3,00","3,00","3,00","2,00","3,00","2,00","3,00","3,00","3,00"


## Vamos a continuar creando los Dataframes con cada uno de los archivos obtenidos de los diferentes indices de sostenibilidad para luego crear los dataframe con toda la informacion 

In [246]:
demanda_agua = '/home/vant/Escritorio/TFM/aguayaire/Demanda total de agua por habitante y dia (l_hab._dia).csv'
df_demanda_agua = pd.read_csv(demanda_agua, sep=';', encoding='latin-1', skiprows= 33)
df_demanda_agua.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2012,2011,2001
0,48001,Abadiño,"173,90","176,02","175,17","176,79","185,59","145,52","203,51"
1,20001,Abaltzisketa,"151,23","154,01","141,22","152,31","155,34","154,65","244,71"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"174,27","176,02","178,02","178,40","183,94","184,19","255,26"


In [247]:
calidad_aire = '/home/vant/Escritorio/TFM/aguayaire/Dias con calidad del aire buena o admisible (%).csv'
df_calidad_aire = pd.read_csv(calidad_aire, sep=';', encoding='latin-1', skiprows= 33)
df_calidad_aire.head(3)

,Código municipio,Municipio,2013,2012,2011,2010,2009,2008,2007,2006,2005
0,48001,Abadiño,"98,90","98,89","98,08","99,73","96,44","92,08","85,48","76,44","87,40"
1,20001,Abaltzisketa,"99,18","99,18","96,99","99,18","96,71","95,63","86,85","86,30","88,74"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"99,45","98,30","97,53","99,73","98,08","95,90","92,88","86,30","91,78"


In [248]:
alumbrado = '/home/vant/Escritorio/TFM/alumbrado.csv'
df_alumbrado = pd.read_csv(alumbrado, sep=';', encoding='latin-1', skiprows= 33)
df_alumbrado.head(3)

,Código municipio,Municipio,2009,2008,2007,2006,2005
0,48001,Abadiño,"185,36","186,64","188,27","182,63","182,05"
1,20001,Abaltzisketa,"269,94","278,48","305,99","164,47","149,15"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"250,51","165,85","161,32","161,31","325,15"


In [249]:
antiguedad_vivienda = '/home/vant/Escritorio/TFM/antiguedadmediavivienda.csv'
df_antiguedad_vivienda = pd.read_csv(antiguedad_vivienda, sep=';', encoding='latin-1', skiprows= 33)
df_antiguedad_vivienda.head(3)

,Código municipio,Municipio,2016,2015,2014,2013,2011,2010,2006,2001,1996,1991
0,48001,Abadiño,"43,90","33,60","32,70","32,30","32,70","33,40","30,87","29,47","26,51","24,32"
1,20001,Abaltzisketa,"52,00","41,60","40,60","39,60","39,10","37,60","32,96","28,36","29,04","77,04"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"88,80","41,90","40,70","39,70","38,60","37,00","35,08","33,33","29,89","34,53"


In [250]:
aseos_publicos = '/home/vant/Escritorio/TFM/aseos.csv'
df_aseos_publicos = pd.read_csv(aseos_publicos, sep=';', encoding='latin-1', skiprows= 33)
df_aseos_publicos[140:142]

#se entiende que es un aseo publico por cada x habitantes

,Código municipio,Municipio,2009,2008,2007,2006,2005
140,1031,Laguardia,"6,61","6,62","6,64","6,74","6,73"
141,1032,Lanciego/Lantziego,"0,00","0,00","0,00","0,00","0,00"


In [251]:
bibliotecas = '/home/vant/Escritorio/TFM/biblios.csv'
df_bibliotecas = pd.read_csv(bibliotecas, sep=';', encoding='latin-1', skiprows= 33)
df_bibliotecas.head(3)

#se entiende que es una biblioteca por cada x habitantes

,Código municipio,Municipio,2017,2016,2015,2014,2013,2011,2010,2009,2008,2004
0,48001,Abadiño,"6,35","6,19","6,04","5,91","5,83","5,55","5,03","4,55","4,26","2,45"
1,20001,Abaltzisketa,"24,52","24,12","23,19","22,44","22,29","20,67","0,00","0,00","0,00","7,14"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"3,42","3,27","3,09","2,98","2,89","2,76","2,67","2,63","1,21","2,28"


In [252]:
bidegorris = '/home/vant/Escritorio/TFM/bidegorris.csv'
df_bidegorris = pd.read_csv(bidegorris, sep=';', encoding='latin-1', skiprows= 33)
df_bidegorris.head(3)

# x kms de bidegorris por cada municipio

,Código municipio,Municipio,2009,2008,2007,2006,2005
0,48001,Abadiño,"0,00","0,00","0,00","0,00","0,00"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"9,77","9,85","7,29","6,24","5,72"


In [253]:
gasto_presup_servicios_sociales = '/home/vant/Escritorio/TFM/bienestar/Gasto corriente del presupuesto municipal per capita en Servicios Sociales ().csv'
df_gasto_presup_servicios_sociales = pd.read_csv(gasto_presup_servicios_sociales, sep=';', encoding='latin-1', skiprows= 10)
df_gasto_presup_servicios_sociales.head(3)

,Código municipio,Municipio,2005,2004,2003,2002
0,48001,Abadiño,"44,28","39,61","25,16","24,66"
1,20001,Abaltzisketa,"49,18","66,14","67,50","49,88"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"92,71","75,09","58,33","57,10"


In [254]:
gasto_percap_serv_soc = '/home/vant/Escritorio/TFM/bienestar/Gasto total per capita en Servicios Sociales ().csv'
df_gasto_percap_serv_soc = pd.read_csv(gasto_percap_serv_soc, sep=';', encoding='latin-1', skiprows= 33)
df_gasto_percap_serv_soc.head(3)

,Código municipio,Municipio,2005,2004,2003,2002
0,48001,Abadiño,"212,71","208,32","169,46","152,29"
1,20001,Abaltzisketa,"261,93","366,07","261,05","218,99"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"480,39","414,24","335,83","317,57"


In [255]:
plazas_res_tercera_edad = '/home/vant/Escritorio/TFM/bienestar/Plazas en centros residenciales para la 3 edad ( habitantes de 65 y mas anos).csv'
df_plazas_res_tercera_edad = pd.read_csv(plazas_res_tercera_edad, sep=';', encoding='latin-1', skiprows= 33)
df_plazas_res_tercera_edad.head(3)

,Código municipio,Municipio,2016,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"56,53","60,04","61,35","61,39","61,47","61,39","60,51","61,63","61,67","41,59"


In [256]:
plazas_res_discapacitadas = '/home/vant/Escritorio/TFM/bienestar/Plazas en centros residenciales para personas discapacitadas ( habitantes).csv'
df_plazas_res_discapacitadas = pd.read_csv(plazas_res_discapacitadas, sep=';', encoding='latin-1', skiprows= 33)
df_plazas_res_discapacitadas.head(3)

,Código municipio,Municipio,2016,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"9,17","9,34","9,44","9,44","9,50","9,59","9,66","9,63","9,68","9,81"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [257]:
grado_ocupacion_residencias = '/home/vant/Escritorio/TFM/bienestar/Grado de ocupacion de los centros residenciales (3 edad y personas con discapacidad) (%).csv'
df_grado_ocupacion_residencias = pd.read_csv(grado_ocupacion_residencias, sep=';', encoding='latin-1', skiprows= 33)
df_grado_ocupacion_residencias.head(3)

,Código municipio,Municipio,2016,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"100,00","98,55","100,00","100,00","100,00","100,00","100,00","100,00","100,00","100,00"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"145,16","84,95","90,32","88,17","86,02","78,49","91,40","96,77","100,00","100,00"


In [258]:
tasa_bruta_natalidad = '/home/vant/Escritorio/TFM/brutanatalidad.csv'
df_tasa_bruta_natalidad = pd.read_csv(tasa_bruta_natalidad, sep=';', encoding='latin-1', skiprows= 33)
df_tasa_bruta_natalidad.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002
0,48001,Abadiño,"9,04","10,23","10,76","10,79","10,96","13,06","12,52","11,49","12,90","14,25","11,72","10,21","10,78","12,71","9,27","9,03"
1,20001,Abaltzisketa,"3,06","6,13","0,00","21,28","0,00","3,10","15,48","6,17","18,69","6,31","19,29","23,33","10,34","17,73","10,68","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"6,29","7,53","6,04","7,45","8,33","8,55","8,44","8,44","9,29","9,76","10,51","9,68","9,00","11,80","10,70","9,40"


In [259]:
aforo_cines = '/home/vant/Escritorio/TFM/calidaddevida/Aforo de cines (butacas por  habitantes).csv'
df_aforo_cines = pd.read_csv(aforo_cines, sep=';', encoding='latin-1', skiprows= 33)
df_aforo_cines.head(3)

,Código municipio,Municipio,2009,2008,2007
0,48001,Abadiño,"0,00","0,00","0,00"
1,20001,Abaltzisketa,"0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"0,00","0,00","0,00"


In [260]:
densidad_poblacional = '/home/vant/Escritorio/TFM/calidaddevida/Densidad poblacional (Hab._Km).csv'
df_densidad_poblacion = pd.read_csv(densidad_poblacional, sep=';', encoding='latin-1', skiprows= 33)
df_densidad_poblacion.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004
0,48001,Abadiño,"208,83","209,13","208,66","208,33","207,05","205,02","202,55","202,49","201,11","199,36","197,78","198,42","197,48","194,76"
1,20001,Abaltzisketa,"28,43","28,20","28,63","28,63","28,80","28,27","28,66","29,11","28,21","28,30","27,14","26,34","25,36","25,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"589,80","591,90","595,49","598,76","603,09","600,00","539,61","600,25","595,49","593,09","593,15","593,09","586,60","575,00"


In [261]:
policial_local = '/home/vant/Escritorio/TFM/calidaddevida/Dotacion policial local ( habitantes).csv'
df_policial_local = pd.read_csv(policial_local, sep=';', encoding='latin-1', skiprows= 33)
df_policial_local.head(3)

,Código municipio,Municipio,2015,2014,2013,2012,2009,2007,2006
0,48001,Abadiño,"1,06","1,06","1,07","0,94","0,55","0,56","0,56"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00","3,15","3,29"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"1,46","1,56","1,55","1,43","1,34","0,94","0,94"


In [262]:
edif_sin_ascensor = '/home/vant/Escritorio/TFM/calidaddevida/Edificios de viviendas de 2 o mas plantas sin ascensor (%).csv'
df_edif_sin_ascensor = pd.read_csv(edif_sin_ascensor, sep=';', encoding='latin-1', skiprows= 10)
df_edif_sin_ascensor.head(3)

,Código municipio,Municipio,2001
0,48001,Abadiño,"88,98"
1,20001,Abaltzisketa,"100,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"94,70"


In [263]:
edif_municip_accesibles = '/home/vant/Escritorio/TFM/calidaddevida/Equipamientos y edificios municipales accesibles (%).csv'
df_edif_municip_accesibles = pd.read_csv(edif_municip_accesibles, sep=';', encoding='latin-1', skiprows= 33)
df_edif_municip_accesibles.head(3)

,Código municipio,Municipio,2009,2008,2007,2006,2005
0,48001,Abadiño,"43,75","43,75","43,75","43,75","43,75"
1,20001,Abaltzisketa,"25,00","25,00","50,00","50,00","50,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"30,77","24,00","23,08","23,08","23,08"


In [264]:
farmacias = '/home/vant/Escritorio/TFM/calidaddevida/Farmacias (por cada 10.000 habitantes).csv'
df_farmacias = pd.read_csv(farmacias, sep=';', encoding='latin-1', skiprows= 33)
df_farmacias.head(3)

,Código municipio,Municipio,2016,2014,2012,2010,2009,2008,2007,2006
0,48001,Abadiño,"3,98","2,66","2,71","2,74","2,74","2,75","2,78","2,80"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"3,13","3,11","3,09","3,09","3,09","3,11","3,12","3,12"


In [265]:
gasolineras = '/home/vant/Escritorio/TFM/calidaddevida/Gasolineras (por cada 10.000 habitantes).csv'
df_gasolineras = pd.read_csv(gasolineras, sep=';', encoding='latin-1', skiprows= 33)
df_gasolineras.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"2,66","2,65","2,66","2,67","2,68","2,71","2,74","2,74","2,75","2,78","2,80","2,79","2,81","2,84","2,86"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"3,14","3,13","3,11","3,10","3,07","3,09","3,09","3,09","3,11","3,12","3,12","4,16","4,21","4,29","4,35"


In [266]:
indice_confort_viv_familias = '/home/vant/Escritorio/TFM/calidaddevida/Indice de confort de las viviendas familiares principales.csv'
df_indice_confort_viv_familias = pd.read_csv(indice_confort_viv_familias, sep=';', encoding='latin-1', skiprows= 33)
df_indice_confort_viv_familias.head(3)

,Código municipio,Municipio,2011,2006,2001,1996,1991
0,48001,Abadiño,"73,80","72,62","69,95","61,00","61,53"
1,20001,Abaltzisketa,"71,60","69,31","67,63","67,00","55,27"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"67,20","70,99","69,19","59,50","58,35"


In [267]:
indice_delitos = '/home/vant/Escritorio/TFM/calidaddevida/Indice de delitos ( habitantes).csv'
df_indice_delitos = pd.read_csv(indice_delitos, sep=';', encoding='latin-1', skiprows= 33)
df_indice_delitos[140:145]

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
140,1031,Laguardia,"67,75","53,52","47,36","72,42","62,38","57,53","73,69","56,25","76,11","70,90","61,54","76,77","60,65","44,53","50,95"
141,1032,Lanciego/Lantziego,"26,59","14,77","29,41","35,29","45,39","49,71","38,01","46,44","33,48","56,93","43,80","50,00","25,49","24,54","34,06"
142,48051,Lanestosa,"3,75","26,22","11,15","14,87","11,19","33,96","3,77","26,02","7,14","17,24","13,79","27,87","17,92","17,73","13,99"
143,1902,Lantarón,"31,32","19,02","35,91","36,18","35,33","51,67","30,14","41,36","32,99","55,73","27,34","27,06","31,19","28,39","20,39"
144,1033,Lapuebla de Labarca,"32,71","21,03","33,88","41,81","46,15","38,46","33,80","38,06","41,28","44,64","22,32","26,68","37,74","29,38","22,17"


In [268]:
indice_dependencia = '/home/vant/Escritorio/TFM/calidaddevida/Indice de dependencia demografica (%).csv'
df_indice_dependencia = pd.read_csv(indice_dependencia, sep=';', encoding='latin-1', skiprows= 33)
df_indice_dependencia.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"63,42","60,82","59,27","56,63","54,63","53,03","49,68","48,67","46,73","44,14","42,86","41,84","43,04","43,34","44,28"
1,20001,Abaltzisketa,"73,94","76,09","79,78","71,35","73,68","70,53","68,95","69,79","69,89","71,35","71,75","68,57","68,05","67,66","65,29"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"55,27","54,52","53,50","52,98","52,23","51,59","51,18","51,42","51,52","50,78","51,35","50,86","51,30","51,96","53,09"


In [269]:
indice_envejecimiento = '/home/vant/Escritorio/TFM/calidaddevida/Indice de envejecimiento: Poblacion de 65 y mas anos (%).csv'
df_indice_envejecimiento = pd.read_csv(indice_envejecimiento, sep=';', encoding='latin-1', skiprows= 33)
df_indice_envejecimiento.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"18,33","17,44","17,12","16,54","15,93","15,48","14,65","14,21","13,71","12,98","12,77","12,62","12,46","12,62","12,52"
1,20001,Abaltzisketa,"18,35","17,59","18,84","17,93","16,67","16,36","17,13","16,87","18,35","19,56","20,07","19,66","20,07","21,07","21,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"17,24","16,97","16,51","16,28","15,87","15,96","15,61","15,58","15,68","15,77","16,00","15,71","15,87","16,52","16,89"


In [270]:
indice_poblacion_infancia14 = '/home/vant/Escritorio/TFM/calidaddevida/Indice de infancia: Poblacion de 0-14 anos (%).csv'
df_indice_poblacion_infancia14 = pd.read_csv(indice_poblacion_infancia14, sep=';', encoding='latin-1', skiprows= 33)
df_indice_poblacion_infancia14.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"16,76","16,83","16,78","16,30","16,04","15,69","14,88","14,49","13,95","13,23","12,66","12,37","12,51","11,96","12,08"
1,20001,Abaltzisketa,"16,82","18,21","19,45","18,54","19,39","20,37","19,94","20,55","18,67","19,56","18,75","17,63","16,20","15,36","14,59"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"13,72","13,96","14,22","14,34","14,52","13,98","14,23","14,18","13,99","13,66","13,41","13,25","13,32","12,95","12,73"


In [271]:
indice_abestecimiento_agua = '/home/vant/Escritorio/TFM/calidaddevida/Indice de rendimiento del sistema de abastecimiento de agua (%).csv'
df_indice_abestecimiento_agua = pd.read_csv(indice_abestecimiento_agua, sep=';', encoding='latin-1', skiprows= 10)
df_indice_abestecimiento_agua.head(3)

,Código municipio,Municipio,2005
0,48001,Abadiño,"43,85"
1,20001,Abaltzisketa,"52,75"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"42,44"


In [272]:
oficinas_bancarias = '/home/vant/Escritorio/TFM/calidaddevida/Oficinas bancarias ( por cada 10.000 habitantes).csv'
df_oficinas_bancarias = pd.read_csv(oficinas_bancarias, sep=';', encoding='latin-1', skiprows= 33)
df_oficinas_bancarias.head(3)

,Código municipio,Municipio,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"9,33","10,73","10,83","10,94","10,94","11,02","11,12","11,20","11,17","11,22","11,38"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"6,19","6,15","6,18","7,21","7,20","7,26","7,29","6,24","6,24","6,31","6,44"


In [273]:
oficinas_correos = '/home/vant/Escritorio/TFM/calidaddevida/Oficinas de Correos (por cada 10.000 habitantes).csv'
df_oficinas_correos = pd.read_csv(oficinas_correos, sep=';', encoding='latin-1', skiprows= 33)
df_oficinas_correos.head(3)

,Código municipio,Municipio,2017,2016,2014,2013,2010,2009,2008,2006
0,48001,Abadiño,"1,33","1,33","1,33","1,34","1,37","1,38","0,00","0,00"
1,20001,Abaltzisketa,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"1,05","1,04","1,03","1,02","1,03","1,04","1,04","0,00"


In [274]:
particip_elec_municip = '/home/vant/Escritorio/TFM/calidaddevida/Participacion en elecciones municipales (% s_censo electoral).csv'
df_particip_elec_municip = pd.read_csv(particip_elec_municip, sep=';', encoding='latin-1', skiprows= 33)
df_particip_elec_municip.head(3)

,Código municipio,Municipio,2015,2011,2007,2003
0,48001,Abadiño,"68,57","71,16","65,94","72,90"
1,20001,Abaltzisketa,"62,45","66,13","55,70","60,39"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"63,44","65,24","63,76","68,34"


In [275]:
poblacion_extranjera = '/home/vant/Escritorio/TFM/calidaddevida/Poblacion extranjera (%).csv'
df_poblacion_extranjera = pd.read_csv(poblacion_extranjera, sep=';', encoding='latin-1', skiprows= 33)
df_poblacion_extranjera.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"4,53","4,86","4,99","4,88","4,97","4,63","4,23","4,49","4,16","3,78","2,87","2,46","1,96","1,15","0,81"
1,20001,Abaltzisketa,"3,98","3,40","2,74","4,26","2,73","2,16","2,80","3,07","1,90","1,89","1,64","0,68","1,06","1,07","0,71"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"1,98","1,80","1,64","1,70","2,07","2,13","2,13","2,01","1,63","1,60","1,34","1,23","1,03","0,89","0,73"


In [276]:
poblacion_total = '/home/vant/Escritorio/TFM/calidaddevida/Poblacion total (hab.).csv'
df_poblacion_total = pd.read_csv(poblacion_total, sep=';', encoding='latin-1', skiprows= 33)
df_poblacion_total.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,1999,1998,1996
0,48001,Abadiño,"7.522,00","7.533,00","7.516,00","7.504,00","7.458,00","7.385,00","7.312,00","7.310,00","7.260,00","7.197,00","7.140,00","7.163,00","7.129,00","7.031,00","6.996,00","6.954,00","6.858,00","6.820,00","6.897,00"
1,20001,Abaltzisketa,"327,00","324,00","329,00","329,00","330,00","324,00","321,00","326,00","316,00","317,00","304,00","295,00","284,00","280,00","281,00","283,00","262,00","265,00","259,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"9.543,00","9.577,00","9.635,00","9.688,00","9.758,00","9.708,00","9.713,00","9.724,00","9.647,00","9.608,00","9.609,00","9.608,00","9.503,00","9.315,00","9.190,00","9.111,00","8.920,00","8.812,00","8.417,00"


In [277]:
puntos_alumbrado_pub = '/home/vant/Escritorio/TFM/calidaddevida/Puntos de alumbrado publico ( habitantes).csv'
df_puntos_alumbrado_pub = pd.read_csv(puntos_alumbrado_pub, sep=';', encoding='latin-1', skiprows= 33)
df_puntos_alumbrado_pub.head(3)

,Código municipio,Municipio,2009,2008,2007,2006,2005
0,48001,Abadiño,"185,36","186,64","188,27","182,63","182,05"
1,20001,Abaltzisketa,"269,94","278,48","305,99","164,47","149,15"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"250,51","165,85","161,32","161,31","325,15"


In [278]:
saldo_migratorio = '/home/vant/Escritorio/TFM/calidaddevida/Saldo migratorio externo ().csv'
df_saldo_migratorio = pd.read_csv(saldo_migratorio, sep=';', encoding='latin-1', skiprows= 33)
df_saldo_migratorio.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"0,80","0,27","0,27","-2,93","-3,61","0,40","6,26","2,60","4,55","7,47","9,76","7,97","5,04","5,65","-1,00"
1,20001,Abaltzisketa,"-3,06","12,35","6,12","0,00","0,00","9,17","6,19","6,17","9,49","9,46","0,00","3,33","0,00","0,00","7,12"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"3,25","2,92","-0,73","0,41","-2,88","4,42","3,60","0,62","0,00","4,05","-0,62","-2,08","1,05","0,11","-1,51"


In [279]:
suelo_urbano = '/home/vant/Escritorio/TFM/calidaddevida/Suelo urbano (%).csv'
df_suelo_urbano = pd.read_csv(suelo_urbano, sep=';', encoding='latin-1', skiprows= 33)
df_suelo_urbano.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004
0,48001,Abadiño,"4,21","4,21","4,21","4,21","4,24","2,44","2,41","2,41","2,41","2,41","2,41","2,41","3,97","3,97"
1,20001,Abaltzisketa,"0,21","0,21","0,21","0,21","0,24","0,25","0,26","0,26","0,26","0,26","0,26","0,26","0,26","0,19"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"14,68","12,12","12,12","12,12","12,12","10,72","9,63","10,43","10,19","10,19","10,19","10,19","10,18","10,20"


In [280]:
superficie_media_viv = '/home/vant/Escritorio/TFM/calidaddevida/Superficie media de las viviendas familiares (m).csv'
df_superficie_media_viv = pd.read_csv(superficie_media_viv, sep=';', encoding='latin-1', skiprows= 33)
df_superficie_media_viv.head(3)

,Código municipio,Municipio,2016,2015,2014,2013,2011,2010,2006,2001
0,48001,Abadiño,"89,70","89,90","89,90","90,00","90,30","90,60","90,64","91,39"
1,20001,Abaltzisketa,"107,20","107,50","107,50","107,50","107,30","109,30","110,75","102,72"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"78,40","78,40","78,20","78,20","78,30","77,80","76,86","75,40"


In [281]:
tasa_crec_vegetativo = '/home/vant/Escritorio/TFM/calidaddevida/Tasa de crecimiento vegetativo ().csv'
df_tasa_crec_vegetativo = pd.read_csv(tasa_crec_vegetativo, sep=';', encoding='latin-1', skiprows= 33)
df_tasa_crec_vegetativo.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002
0,48001,Abadiño,"2,79","3,85","2,52","3,86","4,14","6,60","6,26","4,79","6,04","7,88","3,49","4,61","3,08","6,21","1,14","2,29"
1,20001,Abaltzisketa,"-15,29","3,09","-12,23","18,24","-9,09","-12,23","-3,10","-6,17","3,12","-9,46","12,86","20,00","0,00","0,00","-3,56","-7,09"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"-3,35","-2,61","-6,97","-3,21","-0,62","-1,03","-0,51","-0,41","0,41","1,25","0,73","1,77","-2,30","1,17","-0,11","-2,84"


In [282]:
tiempo_medio_despla_hospital_ref = '/home/vant/Escritorio/TFM/calidaddevida/Tiempo medio de desplazamiento al hospital de referencia (min.).csv'
df_tiempo_medio_despla_hospital_ref = pd.read_csv(tiempo_medio_despla_hospital_ref, sep=';', encoding='latin-1', skiprows= 33)
df_tiempo_medio_despla_hospital_ref.head(3)

,Código municipio,Municipio,2007
0,48001,Abadiño,"12,94"
1,20001,Abaltzisketa,"10,73"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"8,60"


In [283]:
guarderias = '/home/vant/Escritorio/TFM/calidaddevida/Unidades de educacion infantil por cada 100 habitantes de 0 a 2 anos.csv'
df_guarderias = pd.read_csv(guarderias, sep=';', encoding='latin-1', skiprows= 33)
df_guarderias.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004
0,48001,Abadiño,"3,96","4,42","3,36","3,60","3,58","3,12","4,59","3,06","3,41","0,00","4,62","3,25","3,40","1,50"
1,20001,Abaltzisketa,"9,09","11,11","0,00","14,29","0,00","7,69","10,00","7,69","7,69","5,88","6,67","6,67","10,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"3,35","3,35","3,12","3,35","3,53","3,51","3,60","3,50","3,40","3,61","2,78","2,74","2,38","3,11"


In [284]:
variacion_poblac_decada = '/home/vant/Escritorio/TFM/calidaddevida/Variacion de la poblacion en la ultima decada (%).csv'
df_variacion_poblac_decada = pd.read_csv(variacion_poblac_decada, sep=';', encoding='latin-1', skiprows= 33)
df_variacion_poblac_decada.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006
0,48001,Abadiño,"5,35","5,17","5,43","6,73","6,60","6,20","6,88","6,70","5,86","5,53","4,69","3,86"
1,20001,Abaltzisketa,"7,57","9,83","15,85","17,50","17,44","14,49","22,52","26,36","20,61","19,62","14,72","13,90"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"-0,69","-0,32","1,39","4,00","6,18","6,55","7,53","8,18","8,15","9,03","9,04","14,15"


In [285]:
viviendas_mas_50 = '/home/vant/Escritorio/TFM/calidaddevida/Viviendas familiares con mas de 50 anos de antiguedad (%).csv'
df_viviendas_mas_50 = pd.read_csv(viviendas_mas_50, sep=';', encoding='latin-1', skiprows= 33)
df_viviendas_mas_50.head(3)

,Código municipio,Municipio,2016,2015,2014,2013,2011,2010,2006,2001,1991
0,48001,Abadiño,"12,91","11,01","9,03","8,26","8,20","7,60","6,41","5,84","7,83"
1,20001,Abaltzisketa,"25,79","26,11","26,11","25,48","25,16","22,98","19,08","15,70","56,99"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"32,58","31,22","30,48","27,56","26,05","21,09","16,47","12,13","17,89"


In [286]:
viv_desocupadas = '/home/vant/Escritorio/TFM/calidaddevida/Viviendas familiares desocupadas (%).csv'
df_viv_desocupadas = pd.read_csv(viv_desocupadas, sep=';', encoding='latin-1', skiprows= 33)
df_viv_desocupadas.head(3)

,Código municipio,Municipio,2011,2001,1996,1991
0,48001,Abadiño,"13,36","11,24","15,07","14,83"
1,20001,Abaltzisketa,"21,29","31,40","33,98","11,83"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"11,36","10,63","19,92","11,09"


In [287]:
zona_juegos_infantil = '/home/vant/Escritorio/TFM/calidaddevida/Zonas publicas de juegos infantiles ( habitantes de 0 a 14 anos).csv'
df_zona_juegos_infantil = pd.read_csv(zona_juegos_infantil, sep=';', encoding='latin-1', skiprows= 33)
df_zona_juegos_infantil.head(3)

,Código municipio,Municipio,2009,2008,2007,2006,2005
0,48001,Abadiño,"8,50","8,88","9,45",NaN,"10,16"
1,20001,Abaltzisketa,"14,93","16,95","32,26","35,09","38,46"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"18,13","14,81","15,24","3,88","3,93"


In [288]:
viv_certif_energ = '/home/vant/Escritorio/TFM/concienciamedioambiental/Viviendas con certificado de eficiencia energetica  ( ).csv'
df_certif_energ = pd.read_csv(viv_certif_energ, sep=';', encoding='latin-1', skiprows= 33)
df_certif_energ[140:143]

,Código municipio,Municipio,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004
140,1031,Laguardia,"20,83","9,47","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
141,1032,Lanciego/Lantziego,"18,87","5,66","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
142,48051,Lanestosa,"4,95","4,95","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [289]:
pob_estudios_secundarios = '/home/vant/Escritorio/TFM/formacion/Poblacion de mas de 10 anos que ha completado al menos Estudios Secundarios (%).csv'
df_pob_estudios_secundarios = pd.read_csv(pob_estudios_secundarios, sep=';', encoding='latin-1', skiprows= 33)
df_pob_estudios_secundarios.head(3)

,Código municipio,Municipio,2016,2015,2014,2013,2011,2010,2006,2001,1996
0,48001,Abadiño,"59,42","58,78","58,47","57,59","57,13","56,11","43,26","34,82","29,09"
1,20001,Abaltzisketa,"50,18","50,18","48,75","49,10","46,38","43,53","31,03","22,76","20,66"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"61,09","60,05","59,47","58,27","57,36","55,96","44,64","35,43","31,00"


In [290]:
crecimiento_vegetativo = '/home/vant/Escritorio/TFM/crecimientovegetativo.csv'
df_crecimiento_vegetativo = pd.read_csv(crecimiento_vegetativo, sep=';', encoding='latin-1', skiprows= 33)
df_crecimiento_vegetativo.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002
0,48001,Abadiño,"2,79","3,85","2,52","3,86","4,14","6,60","6,26","4,79","6,04","7,88","3,49","4,61","3,08","6,21","1,14","2,29"
1,20001,Abaltzisketa,"-15,29","3,09","-12,23","18,24","-9,09","-12,23","-3,10","-6,17","3,12","-9,46","12,86","20,00","0,00","0,00","-3,56","-7,09"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"-3,35","-2,61","-6,97","-3,21","-0,62","-1,03","-0,51","-0,41","0,41","1,25","0,73","1,77","-2,30","1,17","-0,11","-2,84"


In [291]:
pob_estudios_univer = '/home/vant/Escritorio/TFM/formacion/Poblacion de mas de 10 anos que ha completado estudios Universitarios (%).csv'
df_pob_estudios_univer = pd.read_csv(pob_estudios_univer, sep=';', encoding='latin-1', skiprows= 33)
df_pob_estudios_univer.head(3)

,Código municipio,Municipio,2016,2015,2014,2013,2011,2010,2006,2001,1996
0,48001,Abadiño,"18,22","18,07","18,07","17,56","17,95","16,89","10,81","7,98","4,66"
1,20001,Abaltzisketa,"13,62","13,98","12,81","13,62","13,77","11,51","6,90","5,28","4,55"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"17,39","16,78","16,79","16,39","16,53","16,29","10,98","8,71","5,23"


In [292]:
pob_estudios_profes = '/home/vant/Escritorio/TFM/formacion/Poblacion de mas de 10 anos que ha completado estudios profesionales (%).csv'
df_pob_estudios_profes = pd.read_csv(pob_estudios_profes, sep=';', encoding='latin-1', skiprows= 33)
df_pob_estudios_profes.head(3)

,Código municipio,Municipio,2016,2015,2014,2013,2011,2010,2006,2001,1996
0,48001,Abadiño,"22,59","22,06","22,06","21,77","21,42","19,51","18,21","17,73","14,59"
1,20001,Abaltzisketa,"18,64","16,13","16,01","16,49","16,30","14,75","12,64","10,98","10,74"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"24,26","23,93","23,55","23,01","22,65","20,98","19,08","18,16","15,87"


In [293]:
inver_por_habitante = '/home/vant/Escritorio/TFM/inversionnetaxhabit.csv'
df_inver_por_habitante = pd.read_csv(inver_por_habitante, sep=';', encoding='latin-1', skiprows= 33)
df_inver_por_habitante.head(3)

,Código municipio,Municipio,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002
0,48001,Abadiño,"57,19","107,26","208,83","-77,67","69,09","186,82","133,20","289,99","-41,95","169,49","391,90","294,47","253,47","281,85","748,35"
1,20001,Abaltzisketa,"165,39","183,84","-7,44","717,01","-193,81","112,46","614,44","585,88","316,44","106,32","341,05","371,34","271,93","360,14","708,22"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"28,05","10,82","61,63","45,68","14,62","192,35","23,43","240,10","241,55","242,02","348,27","115,55","236,69","105,90","104,25"


In [294]:
parque_vehiculos = '/home/vant/Escritorio/TFM/parquevehiculos.csv'
df_parque_vehiculos = pd.read_csv(parque_vehiculos, sep=';', encoding='latin-1', skiprows= 33)
df_parque_vehiculos.head(3)

,Código municipio,Municipio,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"0,73","0,73","0,73","0,73","0,74","0,75","0,74","0,73","0,72","0,72","0,68","0,66","0,65","0,63"
1,20001,Abaltzisketa,"0,88","0,88","0,84","0,83","0,84","0,83","0,90","0,90","0,87","0,89","0,88","0,79","0,80","0,78"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"0,76","0,76","0,76","0,76","0,76","0,76","0,76","0,76","0,75","0,73","0,68","0,67","0,66","0,64"


In [295]:
empleo_microempresa = '/home/vant/Escritorio/TFM/tejidoempres/Empleo generado por las microempresas (0-9 empleados) (%).csv'
df_empleo_microempresa = pd.read_csv(empleo_microempresa, sep=';', encoding='latin-1', skiprows= 33)
df_empleo_microempresa.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"20,55","19,69","21,83","22,02","22,93","23,44","23,98","24,80","23,68","23,07","23,09","22,14","20,89","21,75","22,03"
1,20001,Abaltzisketa,"60,18","72,34","50,00","54,76","55,94","46,11","35,92","31,41","32,14","35,62","36,97","48,15","55,71","77,36","100,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"26,64","26,50","27,35","27,59","26,77","25,29","25,90","23,94","23,48","23,29","23,06","23,80","25,38","25,30","25,36"


In [296]:
indice_establ_construc = '/home/vant/Escritorio/TFM/tejidoempres/Porcentaje de establecimientos del sector construccion sobre el total (%).csv'
df_indice_establ_construcc = pd.read_csv(indice_establ_construc, sep=';', encoding='latin-1', skiprows= 33)
df_indice_establ_construcc.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,48001,Abadiño,"12,54","11,53","12,34","12,50","12,24","13,44","15,68","16,34","16,76","16,17","14,99","15,36","13,00","12,97","13,14","13,62","13,89","14,13"
1,20001,Abaltzisketa,"2,70","2,78","3,03","2,94","5,13","10,00","14,81","16,00","16,67","14,81","16,67","13,04","18,75","17,65","21,05","20,00","25,00","18,18"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"17,56","18,06","18,88","18,97","20,29","20,00","20,51","21,24","21,74","21,80","22,54","22,84","22,59","20,51","19,84","19,11","18,86","17,63"


In [297]:
indice_establ_industria = '/home/vant/Escritorio/TFM/tejidoempres/Porcentaje de establecimientos del sector industrial sobre el total (%).csv'
df_indice_establ_industria = pd.read_csv(indice_establ_industria, sep=';', encoding='latin-1', skiprows= 33)
df_indice_establ_industria.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,48001,Abadiño,"17,92","18,18","18,18","18,43","19,00","18,58","19,72","18,89","19,97","19,88","20,24","20,43","21,03","21,88","22,80","22,59","23,03","23,37"
1,20001,Abaltzisketa,"5,41","2,78","3,03","2,94","2,56","2,50","3,70","4,00","4,17","7,41","4,17","13,04","12,50","11,76","10,53","10,00","10,00","13,64"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"13,13","13,51","13,42","13,24","13,43","13,24","13,62","13,80","13,04","13,05","12,56","12,48","13,35","13,97","14,84","14,49","15,19","15,08"


In [298]:
indice_establ_sect_primario = '/home/vant/Escritorio/TFM/tejidoempres/Porcentaje de establecimientos del sector primario sobre el total (%).csv'
df_indice_establ_sect_primario = pd.read_csv(indice_establ_sect_primario, sep=';', encoding='latin-1', skiprows= 33)
df_indice_establ_sect_primario.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012
0,48001,Abadiño,"4,56","5,03","5,19","5,93","6,75","4,83"
1,20001,Abaltzisketa,"35,14","36,11","36,36","35,29","41,03","35,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"1,98","2,12","1,47","2,06","2,19","2,39"


In [299]:
indice_establ_sect_servicios = '/home/vant/Escritorio/TFM/tejidoempres/Porcentaje de establecimientos del sector servicios sobre el total (%).csv'
df_indice_establ_sect_servicios = pd.read_csv(indice_establ_sect_servicios, sep=';', encoding='latin-1', skiprows= 33)
df_indice_establ_sect_servicios.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,48001,Abadiño,"64,98","65,26","64,29","63,14","62,01","63,14","64,60","64,77","63,27","63,95","64,77","64,21","65,97","65,15","64,06","63,79","63,07","62,50"
1,20001,Abaltzisketa,"56,76","58,33","57,58","58,82","51,28","52,50","81,48","80,00","79,17","77,78","79,17","73,91","68,75","70,59","68,42","70,00","65,00","68,18"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"67,33","66,31","66,22","65,74","64,09","64,37","65,87","64,95","65,22","65,15","64,90","64,67","64,06","65,53","65,31","66,40","65,94","67,29"


In [300]:
poblacion_estudia_fuera_munic = '/home/vant/Escritorio/TFM/transporte/Poblacion estudiante de 16 y mas anos que estudia fuera del municipio de residencia (%).csv'
df_poblacion_estudia_fuera_munic = pd.read_csv(poblacion_estudia_fuera_munic, sep=';', encoding='latin-1', skiprows= 33)
df_poblacion_estudia_fuera_munic.head(3)

,Código municipio,Municipio,2011,2001
0,48001,Abadiño,"94,05","95,34"
1,20001,Abaltzisketa,"100,00","100,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"84,89","73,43"


In [301]:
poblacion_trabaja_fuera_munic = '/home/vant/Escritorio/TFM/transporte/Poblacion ocupada de 16 y mas anos que trabaja fuera del municipio de residencia (%).csv'
df_poblacion_trabaja_fuera_munic = pd.read_csv(poblacion_trabaja_fuera_munic, sep=';', encoding='latin-1', skiprows= 33)
df_poblacion_trabaja_fuera_munic.head(3)

,Código municipio,Municipio,2011,2001
0,48001,Abadiño,"66,02","60,57"
1,20001,Abaltzisketa,"74,44","67,48"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"80,36","79,99"


In [302]:
tiempo_medio_acceso_carret = '/home/vant/Escritorio/TFM/transporte/Tiempo medio de acceso a carreteras principales (min.).csv'
df_tiempo_medio_acceso_carret = pd.read_csv(tiempo_medio_acceso_carret, sep=';', encoding='latin-1', skiprows= 10)
df_tiempo_medio_acceso_carret.head(3)

,Código municipio,Municipio,2006
0,48001,Abadiño,"0,88"
1,20001,Abaltzisketa,"6,53"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"0,82"


In [303]:
densidad_comercio_ocasional = '/home/vant/Escritorio/TFM/vitalidadturistica/Densidad comercial de bienes ocasionales ( habitantes).csv'
df_densidad_comercio_ocasional = pd.read_csv(densidad_comercio_ocasional, sep=';', encoding='latin-1', skiprows= 33)
df_densidad_comercio_ocasional.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"5,45","5,71","5,71","5,19","5,48","5,99","6,12","6,84","7,30","6,56","6,56","6,71","6,86","6,78","6,13"
1,20001,Abaltzisketa,"3,07","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","3,22","3,22","3,33","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"2,82","3,12","3,23","3,10","3,81","4,02","4,22","4,22","4,56","4,89","4,37","4,47","3,87","4,04","3,89"


In [304]:
densidad_comercio_minor = '/home/vant/Escritorio/TFM/vitalidadturistica/Densidad comercial minorista ( habitantes).csv'
df_densidad_comercio_minor = pd.read_csv(densidad_comercio_minor, sep=';', encoding='latin-1', skiprows= 33)
df_densidad_comercio_minor.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"8,37","8,77","9,04","8,66","8,96","9,80","9,93","11,08","11,71","10,88","10,04","10,35","10,92","10,45","10,12"
1,20001,Abaltzisketa,"6,13","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","3,22","3,22","3,33","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"6,59","6,87","6,77","6,93","7,71","7,93","8,14","8,85","9,12","9,68","9,26","9,16","9,00","8,61","8,65"


In [305]:
indice_hosteleria = '/home/vant/Escritorio/TFM/vitalidadturistica/Establecimientos de hosteleria y restauracion ( habitantes).csv'
df_indice_hosteleria = pd.read_csv(indice_hosteleria, sep=';', encoding='latin-1', skiprows= 33)
df_indice_hosteleria.head(3)

,Código municipio,Municipio,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,48001,Abadiño,"6,38","6,77","6,64","6,39","5,88","5,26","5,17","5,47","6,06","6,64","6,83","6,99","6,72","7,49","7,13"
1,20001,Abaltzisketa,"12,27","18,40","15,29","18,24","18,18","18,35","15,48","15,43","15,82","15,77","12,86","16,67","17,24","17,73","17,79"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"5,65","5,33","5,62","5,28","4,94","5,55","5,56","5,76","6,22","6,34","6,04","6,45","5,86","6,16","6,38"


In [306]:
plazas_alojamiento_turistico = '/home/vant/Escritorio/TFM/vitalidadturistica/Plazas de alojamiento turistico ( habitantes).csv'
df_plazas_alojamiento_turistico = pd.read_csv(plazas_alojamiento_turistico, sep=';', encoding='latin-1', skiprows= 33)
df_plazas_alojamiento_turistico.head(3)

,Código municipio,Municipio,2017,2010,2009,2008,2007,2006,2005
0,48001,Abadiño,"17,95","20,38","20,38","19,83","19,45","19,61","19,54"
1,20001,Abaltzisketa,"0,00","12,46","12,27","12,66","12,62","13,16","13,56"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"0,42","4,53","4,52","4,56","5,10","5,10","5,10"


In [307]:
superficie_agricola = '/home/vant/Escritorio/TFM/zonasverdes/Superficie agricola (% s_ superficie total).csv'
df_superficie_agricola = pd.read_csv(superficie_agricola, sep=';', encoding='latin-1', skiprows= 33)
df_superficie_agricola.head(3)

,Código municipio,Municipio,2016,2005
0,48001,Abadiño,"1,19","1,16"
1,20001,Abaltzisketa,"1,39","0,18"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"2,10","2,16"


In [308]:
superficie_forestal = '/home/vant/Escritorio/TFM/zonasverdes/Superficie forestal (% s  superficie total).csv'
df_superficie_forestal = pd.read_csv(superficie_forestal, sep=';', encoding='latin-1', skiprows= 33)
df_superficie_forestal.head(3)

,Código municipio,Municipio,2016,2010,2005
0,48001,Abadiño,"53,53","53,27","55,01"
1,20001,Abaltzisketa,"52,43","53,48","50,36"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"35,72","35,68","38,46"


In [309]:
superficie_parques_m2 = '/home/vant/Escritorio/TFM/zonasverdes/Superficie ocupada por parques, jardines y zonas verdes urbanas (%_suelo urbano).csv'
df_superficie_parques_m2 = pd.read_csv(superficie_parques_m2, sep=';', encoding='latin-1', skiprows= 33)
df_superficie_parques_m2.head(3)

,Código municipio,Municipio,2009,2008,2007,2006,2005
0,48001,Abadiño,"8,75","8,75","8,75","8,75","5,30"
1,20001,Abaltzisketa,"103,45","103,45","51,72","3,45","3,46"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"8,60","8,60","8,60","8,60","7,95"


In [310]:
superficie_parques_pers = '/home/vant/Escritorio/TFM/zonasverdes/Superficie ocupada por parques, jardines y zonas verdes urbanas (m_persona).csv'
df_superficie_parques_pers = pd.read_csv(superficie_parques_pers, sep=';', encoding='latin-1', skiprows= 33)
df_superficie_parques_pers.head(3)

,Código municipio,Municipio,2009,2008,2007,2006,2005
0,48001,Abadiño,"10,40","10,47","10,56","10,64","10,61"
1,20001,Abaltzisketa,"92,02","94,94","47,32","3,29","3,39"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"14,60","14,72","14,78","14,78","13,63"


## Ahora hay montar el gran data frame de 2009 con todos los municipios y cada una de las variables (indices de sostenibilidad) seleccionadas para utilizarlas como predictores de las elecciones del 2011.

Al existir NANs (a cada DF le voy a pasar un metodo .isna().sum() para comprobrarlo) , hay que tener cuidado con los diferentes Dataframe, rellenando la informacion con la media.

In [311]:
df_accesibilidad.isna().sum()

Código municipio    0
Municipio           0
2009                4
2008                6
2007                8
2006                1
2005                4
dtype: int64

In [312]:
gran_dataframe = df_accesibilidad[['Código municipio','Municipio','2009']].fillna('59,64')
gran_dataframe['2009'] = gran_dataframe['2009'].str.replace(',', '.').astype(float)
# gran_dataframe.describe()  para saber la media del 2009 y rellenar los NaN con ese dato = 59.64
gran_dataframe[140:142]

,Código municipio,Municipio,2009
140,1031,Laguardia,100.0
141,1032,Lanciego/Lantziego,100.0


## Vamos a seleccionar de cada Dataframe las Columna Municipio y el año 2009 o  mas cercano en caso de no existir

In [313]:
df_accid_t = df_accident_trafico[['Municipio','2009']] 
df_accid_t.head(3)

,Municipio,2009
0,Abadiño,"8,65"
1,Abaltzisketa,"0,00"
2,Abanto y Ciérvana-Abanto Zierbena,"9,60"


## Hacemos un Merge en el Municipio e Inner para obtener solo un Dataframe donde tengamos informacion disponible de todos los municipios 

In [314]:
gdf_2 = pd.merge(gran_dataframe, df_accid_t, on='Municipio', how='inner', suffixes=('_1','_2'))
gdf_2.head(3)

,Código municipio,Municipio,2009_1,2009_2
0,48001,Abadiño,43.75,"8,65"
1,20001,Abaltzisketa,25.00,"0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60"


In [315]:
df_af_cines = df_aforo_cines[['Municipio','2009']]

In [316]:
gdf_3 = pd.merge(gdf_2, df_af_cines, on='Municipio', how='inner')
gdf_3.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009
0,48001,Abadiño,43.75,"8,65","0,00"
1,20001,Abaltzisketa,25.00,"0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00"


In [317]:
df_alumb = df_alumbrado[['Municipio','2009']]

In [318]:
gdf_4 = pd.merge(gdf_3, df_alumb, on='Municipio', how='inner', suffixes=('_3','_4'))
gdf_4.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4
0,48001,Abadiño,43.75,"8,65","0,00","185,36"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51"


In [319]:
df_ant_viv = df_antiguedad_vivienda[['Municipio', '2010']]

In [320]:
gdf_5 = pd.merge(gdf_4, df_ant_viv, on='Municipio', how='inner')
gdf_5.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00"


In [321]:
df_wc_pub = df_aseos_publicos[['Municipio', '2009']]

In [322]:
gdf_6 = pd.merge(gdf_5, df_wc_pub, on='Municipio', how='inner')
gdf_6.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00"


In [323]:
df_bibli = df_bibliotecas[['Municipio', '2009']]

In [324]:
gdf_7 = pd.merge(gdf_6, df_bibli, on='Municipio', how='inner', suffixes=('_6','_7'))
gdf_7.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63"


In [325]:
df_bi_go = df_bidegorris[['Municipio', '2009']]

In [326]:
gdf_8 = pd.merge(gdf_7, df_bi_go, on='Municipio', how='inner')
gdf_8.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77"


In [327]:
df_cld_agua = df_calidad_agua_consumo[['Municipio', '2009']]

In [328]:
gdf_9 = pd.merge(gdf_8, df_cld_agua, on='Municipio', how='inner', suffixes=('_8','_9'))
gdf_9.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00"


In [329]:
df_cld_air = df_calidad_aire[['Municipio', '2009']]

In [330]:
gdf_10 = pd.merge(gdf_9, df_cld_air, on='Municipio', how='inner')
gdf_10.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08"


In [331]:
df_ct_ener = df_certif_energ[['Municipio', '2009']]

In [332]:
gdf_11 = pd.merge(gdf_10, df_ct_ener, on='Municipio', how='inner', suffixes=('_10', '_11'))
gdf_11.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00"


In [333]:
df_crcto_veg = df_crecimiento_vegetativo[['Municipio', '2009']]

In [334]:
gdf_12 = pd.merge(gdf_11, df_crcto_veg, on='Municipio', how='inner')
gdf_12.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41"


In [335]:
df_dmd_agua = df_demanda_agua[['Municipio', '2011']]

In [336]:
gdf_13 = pd.merge(gdf_12, df_dmd_agua, on='Municipio', how='inner')
gdf_13.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009,2011
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19"


In [337]:
df_dens_com_min  = df_densidad_comercio_minor[['Municipio', '2009']]

In [338]:
gdf_14 = pd.merge(gdf_13, df_dens_com_min, on='Municipio', how='inner', suffixes=('_12','_14'))
gdf_14.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12"


In [339]:
df_dens_com_ocas = df_densidad_comercio_ocasional[['Municipio','2009']]

In [340]:
gdf_15 = pd.merge(gdf_14, df_dens_com_ocas, on='Municipio', how='inner')
gdf_15.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56"


In [341]:
df_dens_pob = df_densidad_poblacion[['Municipio', '2009']]

In [342]:
gdf_16 = pd.merge(gdf_15, df_dens_pob, on='Municipio', how='inner', suffixes=('_15','_16'))
gdf_16.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49"


In [343]:
df_ed_sin_asc = df_edif_sin_ascensor[['Municipio', '2001']]

In [344]:
gdf_17 = pd.merge(gdf_16, df_ed_sin_asc, on='Municipio', how='inner')
gdf_17.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70"


In [345]:
df_emp_micro = df_empleo_microempresa[['Municipio', '2009']]

In [346]:
gdf_18 = pd.merge(gdf_17, df_emp_micro, on='Municipio', how='inner')
gdf_18.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48"


In [347]:
df_farma = df_farmacias[['Municipio', '2009']]

In [348]:
gdf_19 = pd.merge(gdf_18, df_farma, on='Municipio', how='inner', suffixes=('_18', '_19'))
gdf_19.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001,2009_18,2009_19
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09"


In [349]:
df_gas = df_gasolineras[['Municipio', '2009']]

In [350]:
gdf_20 = pd.merge(gdf_19, df_gas, on='Municipio', how='inner')
gdf_20.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11"


In [351]:
df_gast_xk_ss = df_gasto_percap_serv_soc[['Municipio', '2005']]

In [352]:
gdf_21 = pd.merge(gdf_20, df_gast_xk_ss, on='Municipio', how='inner')
gdf_21.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009,2005
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39"


In [353]:
df_gast_pres_ss = df_gasto_presup_servicios_sociales[['Municipio', '2005']]

In [354]:
gdf_22 = pd.merge(gdf_21, df_gast_pres_ss, on='Municipio', how='inner', suffixes=('_21','_22'))
gdf_22.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009,2005_21,2005_22
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71"


In [355]:
df_gr_ocu_res = df_grado_ocupacion_residencias[['Municipio','2009']]

In [356]:
gdf_23 = pd.merge(gdf_22, df_gr_ocu_res, on='Municipio', how='inner', suffixes=('_20','_23'))
gdf_23.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17"


In [357]:
df_guarde = df_guarderias[['Municipio', '2009']]

In [358]:
gdf_24 = pd.merge(gdf_23, df_guarde, on='Municipio', how='inner')
gdf_24.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40"


In [359]:
df_ind_abest_agua = df_indice_abestecimiento_agua[['Municipio', '2005']]

In [360]:
gdf_25 = pd.merge(gdf_24, df_ind_abest_agua, on='Municipio', how='inner')
gdf_25.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009,2005
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44"


In [361]:
df_ind_conf_vivfam = df_indice_confort_viv_familias[['Municipio', '2011']]

In [362]:
gdf_26 = pd.merge(gdf_25, df_ind_conf_vivfam, on='Municipio', how='inner', suffixes=('_13', '_26'))
gdf_26.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009,2005,2011_26
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20"


In [363]:
df_indice_delitos['2009'].str.replace(',','.').astype(float).mean() #hay un NaN - 39.25
df_ind_delitos = df_indice_delitos[['Municipio', '2009']].fillna(39.25)

In [364]:
gdf_27 = pd.merge(gdf_26, df_ind_delitos, on='Municipio', how='inner', suffixes=('_24', '_27'))
gdf_27.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90"


In [365]:
df_ind_dep = df_indice_dependencia[['Municipio', '2009']]

In [366]:
gdf_28 = pd.merge(gdf_27, df_ind_dep, on='Municipio', how='inner')
gdf_28.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52"


In [367]:
df_ind_envj = df_indice_envejecimiento[['Municipio','2009']]

In [368]:
gdf_29 = pd.merge(gdf_28, df_ind_envj, on='Municipio', how='inner', suffixes=('_28', '_29'))
gdf_29.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68"


In [369]:
df_ind_est_const = df_indice_establ_construcc[['Municipio','2009']]

In [370]:
gdf_30 = pd.merge(gdf_29, df_ind_est_const, on='Municipio', how='inner')
gdf_30.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74"


In [371]:
df_ind_est_indus = df_indice_establ_industria[['Municipio', '2009']]

In [372]:
gdf_31 = pd.merge(gdf_30, df_ind_est_indus, on='Municipio', how='inner', suffixes=('_30', '_31'))
gdf_31.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04"


In [373]:
df_ind_est_sprim = df_indice_establ_sect_primario[['Municipio', '2012']]

In [374]:
gdf_32 = pd.merge(gdf_31, df_ind_est_sprim, on='Municipio', how='inner')
gdf_32.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39"


In [375]:
df_ind_est_sserv = df_indice_establ_sect_servicios[['Municipio', '2009']]

In [376]:
gdf_33 = pd.merge(gdf_32, df_ind_est_sserv, on='Municipio', how='inner')
gdf_33.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22"


In [377]:
df_ind_host = df_indice_hosteleria[['Municipio', '2009']]

In [378]:
gdf_34 = pd.merge(gdf_33, df_ind_host, on='Municipio', how='inner',suffixes=('_33', '_34'))
gdf_34.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22"


In [379]:
df_ind_inf14 = df_indice_poblacion_infancia14[['Municipio','2009']]

In [380]:
gdf_35 = pd.merge(gdf_34, df_ind_inf14, on='Municipio', how='inner')
gdf_35.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99"


In [381]:
df_inv_x_hab = df_inver_por_habitante[['Municipio', '2009']]

In [382]:
gdf_36 = pd.merge(gdf_35, df_inv_x_hab, on='Municipio', how='inner',suffixes=('_35', '_36'))
gdf_36.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10"


In [383]:
df_of_banco = df_oficinas_bancarias[['Municipio', '2009']]

In [384]:
gdf_37 = pd.merge(gdf_36, df_of_banco, on='Municipio', how='inner')
gdf_37.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20"


In [385]:
df_of_correos = df_oficinas_correos[['Municipio', '2009']]

In [386]:
gdf_38 = pd.merge(gdf_37, df_of_correos, on='Municipio', how='inner',suffixes=('_37', '_38'))
gdf_38.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04"


In [387]:
df_pq_vehic = df_parque_vehiculos[['Municipio', '2009']]

In [388]:
gdf_39 = pd.merge(gdf_38, df_pq_vehic, on='Municipio', how='inner')
gdf_39.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76"


In [389]:
df_plazas_aloj_turis = df_plazas_alojamiento_turistico[['Municipio','2009']]

In [390]:
gdf_40 = pd.merge(gdf_39, df_plazas_aloj_turis, on='Municipio', how='inner',suffixes=('_39', '_40'))
gdf_40.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52"


In [391]:
df_plazas_resi_disca = df_plazas_res_discapacitadas[['Municipio', '2009']]

In [392]:
gdf_41 = pd.merge(gdf_40, df_plazas_resi_disca, on='Municipio', how='inner')
gdf_41.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00"


In [393]:
df_plazas_resi_3edad = df_plazas_res_tercera_edad[['Municipio', '2009']]

In [394]:
gdf_42 = pd.merge(gdf_41, df_plazas_resi_3edad, on='Municipio', how='inner', suffixes=('_41', '_42'))
gdf_42.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39"


In [395]:
df_pob_est_prof = df_pob_estudios_profes[['Municipio', '2010']]

In [396]:
gdf_43 = pd.merge(gdf_42, df_pob_est_prof, on='Municipio', how='inner', suffixes=('_5', '_43'))
gdf_43.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98"


In [397]:
df_pob_est_2 = df_pob_estudios_secundarios[['Municipio', '2010']]

In [398]:
gdf_44 = pd.merge(gdf_43, df_pob_est_2, on='Municipio', how='inner')
gdf_44.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96"


In [399]:
df_pob_est_univ = df_pob_estudios_univer[['Municipio', '2010']]

In [400]:
gdf_45 = pd.merge(gdf_44, df_pob_est_univ, on='Municipio', how='inner', suffixes=('_44', '_45'))
gdf_45.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29"


In [401]:
df_pob_est_fuera_mu = df_poblacion_estudia_fuera_munic[['Municipio', '2011']]

In [402]:
gdf_46 = pd.merge(gdf_45, df_pob_est_fuera_mu, on='Municipio', how='inner')
gdf_46.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89"


In [403]:
df_pob_extranj = df_poblacion_extranjera[['Municipio', '2009']]

In [404]:
gdf_47 = pd.merge(gdf_46, df_pob_extranj, on='Municipio', how='inner')
gdf_47.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63"


In [405]:
df_pob_tot = df_poblacion_total[['Municipio', '2009']]

In [406]:
gdf_48 = pd.merge(gdf_47, df_pob_tot, on='Municipio', how='inner', suffixes=('_47', '_48'))
gdf_48.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011,2009_47,2009_48
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00"


In [407]:
df_pob_trabj_fuera_mu = df_poblacion_trabaja_fuera_munic[['Municipio', '2011']]

In [408]:
pd.options.display.max_columns= None
gdf_49 = pd.merge(gdf_48, df_pob_trabj_fuera_mu, on='Municipio', how='inner', suffixes=('_46', '_49'))
gdf_49.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36"


In [409]:
df_poli_local = df_policial_local[['Municipio', '2009']]

In [410]:
gdf_50 = pd.merge(gdf_49, df_poli_local, on='Municipio', how='inner')
gdf_50.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34"


In [411]:
df_saldo_migra = df_saldo_migratorio[['Municipio', '2009']]

In [412]:
gdf_51 = pd.merge(gdf_50, df_saldo_migra, on='Municipio', how='inner', suffixes=('_50', '_51'))
gdf_51.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00"


In [413]:
df_suelo_urba = df_suelo_urbano[['Municipio', '2009']]

In [414]:
gdf_52 = pd.merge(gdf_51, df_suelo_urba, on='Municipio', how='inner')
gdf_52.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19"


In [415]:
df_sup_agric = df_superficie_agricola[['Municipio', '2005']]

In [416]:
gdf_53 = pd.merge(gdf_52, df_sup_agric, on='Municipio', how='inner', suffixes=('_25', '_53'))
gdf_53.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009,2005_53
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16"


In [417]:
df_sup_forest = df_superficie_forestal[['Municipio', '2005']]

In [418]:
gdf_54 = pd.merge(gdf_53, df_sup_forest, on='Municipio', how='inner')
gdf_54.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009,2005_53,2005
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46"


In [419]:
df_sup_media_viv = df_superficie_media_viv[['Municipio','2006']]

In [420]:
gdf_55 = pd.merge(gdf_54, df_sup_media_viv, on='Municipio', how='inner')
gdf_55.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009,2005_53,2005,2006
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86"


In [421]:
df_superficie_parques_m2['2009'].str.replace(',','.').astype(float).mean() #hay un NaN - 19.96
df_sup_parq_m2 = df_superficie_parques_m2[['Municipio', '2009']].fillna(19.96)

In [422]:
gdf_56 = pd.merge(gdf_55, df_sup_parq_m2, on='Municipio', how='inner', suffixes=('_52', '_56'))
gdf_56.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006,2009_56
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60"


In [423]:
df_superficie_parques_pers['2009'].str.replace(',','.').astype(float).mean() #hay un NaN - 36.58
df_sup_parq_perso = df_superficie_parques_pers[['Municipio', '2009']].fillna(36.58)

In [424]:
gdf_57 = pd.merge(gdf_56, df_sup_parq_perso, on='Municipio', how='inner')
gdf_57.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006,2009_56,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75","10,40"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45","92,02"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60","14,60"


In [425]:
df_tasa_brt_natalid = df_tasa_bruta_natalidad[['Municipio', '2009']]

In [426]:
gdf_58 = pd.merge(gdf_57, df_tasa_brt_natalid, on='Municipio', how='inner', suffixes=('_57', '_58'))
gdf_58.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006,2009_56,2009_57,2009_58
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75","10,40","12,90"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45","92,02","18,69"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60","14,60","9,29"


In [427]:
df_tiempo_med_acc_carret = df_tiempo_medio_acceso_carret[['Municipio', '2006']]

In [428]:
gdf_59 = pd.merge(gdf_58, df_tiempo_med_acc_carret, on='Municipio', how='inner', suffixes=('_55','_59'))
gdf_59.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006_55,2009_56,2009_57,2009_58,2006_59
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75","10,40","12,90","0,88"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45","92,02","18,69","6,53"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60","14,60","9,29","0,82"


In [429]:
df_tiempo_med_despl_hosp_ref = df_tiempo_medio_despla_hospital_ref[['Municipio', '2007']]

In [430]:
gdf_60 = pd.merge(gdf_59, df_tiempo_med_despl_hosp_ref, on='Municipio', how='inner')
gdf_60.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006_55,2009_56,2009_57,2009_58,2006_59,2007
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75","10,40","12,90","0,88","12,94"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45","92,02","18,69","6,53","10,73"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60","14,60","9,29","0,82","8,60"


In [431]:
df_variacion_poblac_decada['2009'].str.replace(',','.').astype(float).mean() #hay un NaN - 13.55
df_var_pob_10 = df_variacion_poblac_decada[['Municipio', '2009']].fillna(13.55)

In [432]:
gdf_61 = pd.merge(gdf_60, df_var_pob_10, on='Municipio', how='inner')
gdf_61.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006_55,2009_56,2009_57,2009_58,2006_59,2007,2009
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75","10,40","12,90","0,88","12,94","5,86"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45","92,02","18,69","6,53","10,73","20,61"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60","14,60","9,29","0,82","8,60","8,15"


In [433]:
df_viv_desocu = df_viv_desocupadas[['Municipio', '2001']]

In [434]:
gdf_62 = pd.merge(gdf_61, df_viv_desocu, on='Municipio', how='inner', suffixes=('_17', '_62'))
gdf_62.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001_17,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006_55,2009_56,2009_57,2009_58,2006_59,2007,2009,2001_62
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75","10,40","12,90","0,88","12,94","5,86","11,24"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45","92,02","18,69","6,53","10,73","20,61","31,40"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60","14,60","9,29","0,82","8,60","8,15","10,63"


In [435]:
df_viv_mas_50 = df_viviendas_mas_50[['Municipio', '2010']]

In [436]:
gdf_63 = pd.merge(gdf_62, df_viv_mas_50, on='Municipio', how='inner')
gdf_63.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001_17,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006_55,2009_56,2009_57,2009_58,2006_59,2007,2009,2001_62,2010
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75","10,40","12,90","0,88","12,94","5,86","11,24","7,60"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45","92,02","18,69","6,53","10,73","20,61","31,40","22,98"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60","14,60","9,29","0,82","8,60","8,15","10,63","21,09"


In [437]:
df_zona_jueg_inf = df_zona_juegos_infantil[['Municipio','2009']]

In [438]:
gdf_64 = pd.merge(gdf_63, df_zona_jueg_inf, on='Municipio', how='inner',suffixes=('_61', '_64'))
gdf_64.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001_17,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006_55,2009_56,2009_57,2009_58,2006_59,2007,2009_61,2001_62,2010,2009_64
0,48001,Abadiño,43.75,"8,65","0,00","185,36","33,40","0,00","4,55","0,00","2,00","96,44","0,00","6,04","145,52","11,71","7,30","201,11","88,98","23,68","2,74","2,75","212,71","44,28","100,00","3,41","43,85","73,80","49,69","46,73","13,71","16,76","19,97","4,83","63,27","6,06","13,95","289,99","10,94","1,38","0,73","20,38","9,44","0,00","19,51","56,11","16,89","94,05","4,16","7.260,00","66,02","0,55","4,55","2,41","1,16","55,01","90,64","8,75","10,40","12,90","0,88","12,94","5,86","11,24","7,60","8,50"
1,20001,Abaltzisketa,25.00,"0,00","0,00","269,94","37,60","0,00","0,00","0,00","3,00","96,71","0,00","3,12","154,65","0,00","0,00","28,21","100,00","32,14","0,00","0,00","261,93","49,18","0,00","7,69","52,75","71,60","43,61","69,89","18,35","16,67","4,17","35,00","79,17","15,82","18,67","585,88","0,00","0,00","0,90","12,27","0,00","0,00","14,75","43,53","11,51","100,00","1,90","316,00","74,44","0,00","9,49","0,26","0,18","50,36","110,75","103,45","92,02","18,69","6,53","10,73","20,61","31,40","22,98","14,93"
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,"9,60","0,00","250,51","37,00","0,00","2,63","9,77","3,00","98,08","0,00","0,41","184,19","9,12","4,56","595,49","94,70","23,48","3,09","3,11","480,39","92,71","88,17","3,40","42,44","67,20","47,90","51,52","15,68","21,74","13,04","2,39","65,22","6,22","13,99","240,10","7,20","1,04","0,76","4,52","0,00","61,39","20,98","55,96","16,29","84,89","1,63","9.647,00","80,36","1,34","0,00","10,19","2,16","38,46","76,86","8,60","14,60","9,29","0,82","8,60","8,15","10,63","21,09","18,13"


## Muchos de las variables, casi todas, son strings por lo que no van a ser muy utiles para hacer modelos de prediccion. Asi que generamos una funcion que convierta los string en float.

In [439]:
def to_float_from_str_decimal(string):
    """ It convert '12,5' to 12.5 """
    try:
        return float(string.replace(',', '.'))
    except:
        return float(string.replace('.','')[:-3])
    
to_float_from_str_decimal('1.900,00')

1900.0

In [440]:
#2009_2,2009_3,2009_4,2010_5,2009_6,2009_7 ,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16
#2001_17,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30
#2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44
#2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006_55,2009_56,2009_57,2009_58
#2006_59,2007,2009_61,2001_62,2010,2009_64 

In [441]:
gdf_64.to_csv('convertir2009.csv')

In [442]:
F_GDF = pd.read_csv('../TFM/convertir2009.csv',
                   converters={'2009_2': to_float_from_str_decimal,
                                             '2009_3': to_float_from_str_decimal,
                                             '2009_4': to_float_from_str_decimal,
                                             '2010_5': to_float_from_str_decimal,
                                             '2009_6': to_float_from_str_decimal,
                                             '2009_7': to_float_from_str_decimal,
                                             '2009_8': to_float_from_str_decimal,
                                             '2009_9': to_float_from_str_decimal,
                                             '2009_10': to_float_from_str_decimal,
                                             '2009_11': to_float_from_str_decimal,
                                             '2009_12': to_float_from_str_decimal,
                                             '2011_13': to_float_from_str_decimal,
                                             '2009_14': to_float_from_str_decimal,
                                             '2009_15': to_float_from_str_decimal,
                                             '2009_16': to_float_from_str_decimal,
                                              '2001_17': to_float_from_str_decimal,
                                             '2009_18': to_float_from_str_decimal,
                                             '2009_19': to_float_from_str_decimal,
                                             '2009_20': to_float_from_str_decimal,
                                             '2005_21': to_float_from_str_decimal,
                                             '2005_22': to_float_from_str_decimal,
                                             '2009_23': to_float_from_str_decimal,
                                             '2009_24': to_float_from_str_decimal,
                                             '2005_25': to_float_from_str_decimal,
                                             '2011_26': to_float_from_str_decimal,
                                             '2009_27': to_float_from_str_decimal,
                                             '2009_28': to_float_from_str_decimal,
                                             '2009_29': to_float_from_str_decimal,
                                             '2009_30': to_float_from_str_decimal,
                                             '2009_31': to_float_from_str_decimal,
                                             '2012': to_float_from_str_decimal,
                                             '2009_33': to_float_from_str_decimal,
                                             '2009_34': to_float_from_str_decimal,
                                             '2009_35': to_float_from_str_decimal,
                                             '2009_36': to_float_from_str_decimal,
                                             '2009_37': to_float_from_str_decimal,
                                             '2009_38': to_float_from_str_decimal,
                                             '2009_39': to_float_from_str_decimal,
                                             '2009_40': to_float_from_str_decimal,
                                             '2009_41': to_float_from_str_decimal,
                                             '2009_42': to_float_from_str_decimal,
                                             '2010_43': to_float_from_str_decimal,
                                             '2010_44': to_float_from_str_decimal,
                                             '2010_45': to_float_from_str_decimal,
                                             '2011_46': to_float_from_str_decimal,
                                             '2009_47': to_float_from_str_decimal,
                                             '2009_48': to_float_from_str_decimal,
                                             '2011_49': to_float_from_str_decimal,
                                             '2009_50': to_float_from_str_decimal,
                                             '2009_51': to_float_from_str_decimal,
                                             '2009_52': to_float_from_str_decimal,
                                             '2005_53': to_float_from_str_decimal,
                                             '2005': to_float_from_str_decimal,
                                             '2006_55': to_float_from_str_decimal,
                                             '2009_56': to_float_from_str_decimal,
                                             '2009_57': to_float_from_str_decimal,
                                             '2009_58': to_float_from_str_decimal,
                                             '2006_59': to_float_from_str_decimal,
                                             '2007': to_float_from_str_decimal,
                                             '2009_61': to_float_from_str_decimal,
                                             '2001_62': to_float_from_str_decimal,
                                             '2010': to_float_from_str_decimal,
                                             '2009_64': to_float_from_str_decimal,})
F_GDF_1 = F_GDF.drop('Unnamed: 0', axis=1)

## Hasta ahora hemos ido sumando predictores/variables/indices de sostenibilidad al Dataframe del 2009 con todos los municipios con informacion disponible. 

Ahora crearemos un nuevo Dataframe solo con la informacion de participacion en las elecciones municipales 

In [443]:
df_particip_elec_municip.head(3)

,Código municipio,Municipio,2015,2011,2007,2003
0,48001,Abadiño,"68,57","71,16","65,94","72,90"
1,20001,Abaltzisketa,"62,45","66,13","55,70","60,39"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"63,44","65,24","63,76","68,34"


In [444]:
df_0 = df_particip_elec_municip[['Código municipio','Municipio', '2003', '2007', '2011', '2015']]

In [445]:
df_0['2015'].str.replace(',', '.').astype(float).describe()
#median 2003 - %77.09
#median 2007 - %70.28
#median 2011 - %73.13
#median 2015 - %69.87
#average - %72.59

count    250.000000
mean      71.020680
std        8.405047
min       31.370000
25%       64.377500
50%       69.875000
75%       78.095000
max       91.530000
Name: 2015, dtype: float64

## Tenemos la media de las medianas de todas las utimas elecciones, la utilizaremos de referencia para crear una etiqueta para nuestro clasificador

In [446]:
df_0['2011'] = df_0['2011'].str.replace(',', '.').astype(float)

In [447]:
df_0.head(3)

,Código municipio,Municipio,2003,2007,2011,2015
0,48001,Abadiño,"72,90","65,94",71.16,"68,57"
1,20001,Abaltzisketa,"60,39","55,70",66.13,"62,45"
2,48002,Abanto y Ciérvana-Abanto Zierbena,"68,34","63,76",65.24,"63,44"


In [448]:
#He creado una funcion para generar un label que añadire al dataframe que etiqueta los municipios en base 
#a su nivel de participacion ciudadana en las elecciones municipales, tomando como referencia la mediana que no la
#media ya que sirve en este caso mejor de referencia para evitar datos outliers de poblaciones pequeñas. 
def label_partic(participacion):
    if participacion > 72.59:
        return 1
    else:
        return 2
label_partic(50)

2

In [449]:
df_0['2011_partic'] = df_0['2011'].apply(label_partic)

In [450]:
df_0.head(3)

,Código municipio,Municipio,2003,2007,2011,2015,2011_partic
0,48001,Abadiño,"72,90","65,94",71.16,"68,57",2
1,20001,Abaltzisketa,"60,39","55,70",66.13,"62,45",2
2,48002,Abanto y Ciérvana-Abanto Zierbena,"68,34","63,76",65.24,"63,44",2


In [451]:
df_clasif_2011 = df_0[['Municipio','2011', '2011_partic']]
df_clasif_2011.columns = ['Municipio', '2011_elecc', '2011_partic']
df_clasif_2011.head(3)

,Municipio,2011_elecc,2011_partic
0,Abadiño,71.16,2
1,Abaltzisketa,66.13,2
2,Abanto y Ciérvana-Abanto Zierbena,65.24,2


In [452]:
gdf_65 = pd.merge(F_GDF_1, df_clasif_2011, on='Municipio', how='inner')
gdf_65.head(3)

,Código municipio,Municipio,2009_1,2009_2,2009_3,2009_4,2010_5,2009_6,2009_7,2009_8,2009_9,2009_10,2009_11,2009_12,2011_13,2009_14,2009_15,2009_16,2001_17,2009_18,2009_19,2009_20,2005_21,2005_22,2009_23,2009_24,2005_25,2011_26,2009_27,2009_28,2009_29,2009_30,2009_31,2012,2009_33,2009_34,2009_35,2009_36,2009_37,2009_38,2009_39,2009_40,2009_41,2009_42,2010_43,2010_44,2010_45,2011_46,2009_47,2009_48,2011_49,2009_50,2009_51,2009_52,2005_53,2005,2006_55,2009_56,2009_57,2009_58,2006_59,2007,2009_61,2001_62,2010,2009_64,2011_elecc,2011_partic
0,48001,Abadiño,43.75,8.65,0.0,185.36,33.4,0.0,4.55,0.00,2.0,96.44,0.0,6.04,145.52,11.71,7.30,201.11,88.98,23.68,2.74,2.75,212.71,44.28,100.00,3.41,43.85,73.8,49.69,46.73,13.71,16.76,19.97,4.83,63.27,6.06,13.95,289.99,10.94,1.38,0.73,20.38,9.44,0.00,19.51,56.11,16.89,94.05,4.16,7260.0,66.02,0.55,4.55,2.41,1.16,55.01,90.64,8.75,10.40,12.90,0.88,12.94,5.86,11.24,7.60,8.50,71.16,2
1,20001,Abaltzisketa,25.00,0.00,0.0,269.94,37.6,0.0,0.00,0.00,3.0,96.71,0.0,3.12,154.65,0.00,0.00,28.21,100.00,32.14,0.00,0.00,261.93,49.18,0.00,7.69,52.75,71.6,43.61,69.89,18.35,16.67,4.17,35.00,79.17,15.82,18.67,585.88,0.00,0.00,0.90,12.27,0.00,0.00,14.75,43.53,11.51,100.00,1.90,316.0,74.44,0.00,9.49,0.26,0.18,50.36,110.75,103.45,92.02,18.69,6.53,10.73,20.61,31.40,22.98,14.93,66.13,2
2,48002,Abanto y Ciérvana-Abanto Zierbena,30.77,9.60,0.0,250.51,37.0,0.0,2.63,9.77,3.0,98.08,0.0,0.41,184.19,9.12,4.56,595.49,94.70,23.48,3.09,3.11,480.39,92.71,88.17,3.40,42.44,67.2,47.90,51.52,15.68,21.74,13.04,2.39,65.22,6.22,13.99,240.10,7.20,1.04,0.76,4.52,0.00,61.39,20.98,55.96,16.29,84.89,1.63,9647.0,80.36,1.34,0.00,10.19,2.16,38.46,76.86,8.60,14.60,9.29,0.82,8.60,8.15,10.63,21.09,18.13,65.24,2


## Para que todo el mundo tenga ya accesible el Dataframe, limpio y ordenado, vamos a subirlo a un archivo csv con el nombre de GDataFrame_2009.csv

In [453]:
gdf_65.to_csv('GDataframe_2009.csv')